TAREA: Los ejemplos ilustrativos anteriores permiten saber el número de monedas presentes en la imagen. ¿Cómo saber la cantidad de dinero presente en ella? Sugerimos identificar de forma interactiva (por ejemplo haciendo clic en la imagen) una moneda de un valor determinado en la imagen (por ejemplo de 1€). Tras obtener esa información y las dimensiones en milímetros de las distintas monedas, realiza una propuesta para estimar la cantidad de dinero en la imagen. Muestra la cuenta de monedas y dinero sobre la imagen. No hay restricciones sobre utilizar medidas geométricas o de color.

Una vez resuelto el reto con la imagen ideal proporcionada, captura una o varias imágenes con monedas. Aplica el mismo esquema, tras identificar la moneda del valor determinado, calcula el dinero presente en la imagen. ¿Funciona correctamente? ¿Se observan problemas?

Nota: Para establecer la correspondencia entre píxeles y milímetros, comentar que la moneda de un euro tiene un diámetro de 23.25 mm. la de 50 céntimos de 24.35, la de 20 céntimos de 22.25, etc.

Extras: Considerar que la imagen pueda contener objetos que no son monedas y/o haya solape entre las monedas. Demo en vivo.

In [9]:
import cv2
import numpy as np

tipo_de_moneda = ["2e", "1e", "50c", "20c", "10c", "5c", "2c", "1c"]
proporciones_originales = [1.11, 1, 1.04, 0.96, 0.85, 0.91, 0.81, 0.7]

valores_euros = {
    "2e": 2.00, "1e": 1.00, "50c": 0.50, "20c": 0.20,
    "10c": 0.10, "5c": 0.05, "2c": 0.02, "1c": 0.01
}


def lista_proporcional(tipo):
    pos = tipo_de_moneda.index(tipo)
    valor = proporciones_originales[pos]
    return [i / valor for i in proporciones_originales]

def hallar_valor(diametro_ref, diametro_nuevo, proporciones):
    proporcion_calculada = diametro_nuevo / diametro_ref
    mejor_dif = float('inf')
    pos = 0
    for i, propor in enumerate(proporciones):
        dif = abs(proporcion_calculada - propor)
        if dif < mejor_dif:
            mejor_dif = dif
            pos = i
    return tipo_de_moneda[pos]

def calcular_diametro(cnt):
    (x, y), radius = cv2.minEnclosingCircle(cnt)
    return radius * 2



img = cv2.imread('Monedas.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (7, 7), 0)
_, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)


kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)
thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=3)


contornos, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contornos_filtrados = [c for c in contornos if 10000 < cv2.contourArea(c) < 200000]


centros = []
contornos_finales = []
for c in contornos_filtrados:
    (x, y), _ = cv2.minEnclosingCircle(c)
    centro = np.array([x, y])
    if all(np.linalg.norm(centro - np.array(p)) > 30 for p in centros):
        contornos_finales.append(c)
        centros.append(centro)

contornos = contornos_finales
print(f"Contornos finales: {len(contornos)}")


output = img.copy()
monedas_asignadas = {}
ref_diametro = None
proporciones = lista_proporcional("1c")

for i, c in enumerate(contornos):
    (x, y), _ = cv2.minEnclosingCircle(c)
    cv2.putText(output, str(i), (int(x) - 10, int(y)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
    cv2.drawContours(output, [c], -1, (0, 255, 0), 2)

cv2.namedWindow("Monedas")
cv2.imshow("Monedas", output)


def click_event(event, x, y, flags, param):
    global ref_diametro, monedas_asignadas, output

    if event == cv2.EVENT_LBUTTONDOWN:
        for i, contorno in enumerate(contornos):
            if cv2.pointPolygonTest(contorno, (x, y), False) >= 0:
                print(f"\nClick sobre moneda #{i}")

                tipo = input(f"Tipo de moneda ({', '.join(tipo_de_moneda)}): ").strip()
                if tipo not in tipo_de_moneda:
                    print("Moneda no válida.")
                    return

                ref_diametro = calcular_diametro(contorno)
                print(f"Referencia: {tipo}, diámetro = {ref_diametro:.2f}px")

                proporciones_ref = lista_proporcional(tipo)
                monedas_asignadas.clear()
                output = img.copy()

                for j, c in enumerate(contornos):
                    diam = calcular_diametro(c)
                    tipo_estimado = hallar_valor(ref_diametro, diam, proporciones_ref)
                    monedas_asignadas[j] = tipo_estimado

                    M = cv2.moments(c)
                    if M["m00"] != 0:
                        cx = int(M["m10"] / M["m00"])
                        cy = int(M["m01"] / M["m00"])
                        cv2.putText(output, tipo_estimado, (cx - 25, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                        cv2.drawContours(output, [c], -1, (0, 255, 0), 2)

                contar_monedas_y_valor()
                cv2.imshow("Monedas", output)
                break

def contar_monedas_y_valor():
    conteo = {}
    total = 0
    for tipo in monedas_asignadas.values():
        conteo[tipo] = conteo.get(tipo, 0) + 1
        total += valores_euros[tipo]

    print("\n--- RESULTADOS ---")
    for tipo, cantidad in conteo.items():
        print(f"{tipo}: {cantidad} moneda(s)")
    print(f"Total estimado: {total:.2f} €")

cv2.setMouseCallback("Monedas", click_event)


while True:
    key = cv2.waitKey(1) & 0xFF
    if key == 27:
        break

cv2.destroyAllWindows()


print("\nMonedas identificadas:")
for idx, tipo in monedas_asignadas.items():
    print(f"Moneda #{idx}: {tipo}")


Contornos finales: 8

Click sobre moneda #7
Referencia: 2e, diámetro = 211.25px

--- RESULTADOS ---
1c: 1 moneda(s)
20c: 1 moneda(s)
2c: 1 moneda(s)
50c: 1 moneda(s)
10c: 1 moneda(s)
1e: 1 moneda(s)
5c: 1 moneda(s)
2e: 1 moneda(s)
Total estimado: 3.88 €

Monedas identificadas:
Moneda #0: 1c
Moneda #1: 20c
Moneda #2: 2c
Moneda #3: 50c
Moneda #4: 10c
Moneda #5: 1e
Moneda #6: 5c
Moneda #7: 2e


TAREA: La tarea consiste en extraer características (geométricas y/o visuales) de las tres imágenes completas de partida, y aprender patrones que permitan identificar las partículas en nuevas imágenes. Para ello se proporciona como imagen de test MPs_test.jpg y sus correpondientes anotaciones MPs_test_bbs.csv con la que deben obtener las métricas para su propuesta de clasificación de microplásticos, además de la matriz de confusión. La matriz de confusión permitirá mostrar para cada clase el número de muestras que se clasifican correctamente de dicha clase, y el número de muestras que se clasifican incorrectamente como perteneciente a una de las otras dos clases